In [4]:
import os
import dotenv
from pathlib import Path

from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import (
    WebBaseLoader, 
    PyPDFLoader, 
    Docx2txtLoader,
)

from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

dotenv.load_dotenv()

USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [5]:
#Load the documents
doc_paths = [
    "docs/test_rag.pdf",
    "docs/test_rag.docx"
]

docs = []
for doc_file in doc_paths:
    file_path = Path(doc_file)
    
    try:
        if doc_file.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif doc_file.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        elif doc_file.endswith('.txt') or doc_file.endswith(".md"):
            loader = TextLoader(file_path)
        else:
            print(f"Document type {doc_file.type} not supported")
            continue

        docs.extend(loader.load())

    except Exception as e:
        print(f"Error loading document {doc_file}: {e}")


#Load from url
url = 'https://docs.streamlit.io/develop/quick-reference/release-notes'
try:
    loader = WebBaseLoader(url)
    docs.extend(loader.load())
except Exception as e:
    print(f"Error loading document from {url}: {e}")


In [6]:
docs

[Document(metadata={'source': 'docs\\test_rag.pdf', 'page': 0}, page_content='My favorite food is margarita pizza.  \nThere are 47588 bottles in the tr uck.  '),
 Document(metadata={'source': 'docs\\test_rag.docx'}, page_content='My favorite food is margarita pizza.\n\nThere are 47588 bottles in the truck.'),
 Document(metadata={'source': 'https://docs.streamlit.io/develop/quick-reference/release-notes', 'title': 'Release notes - Streamlit Docs', 'description': 'A changelog of highlights and fixes for each version of Streamlit.', 'language': 'No language found.'}, page_content='Release notes - Streamlit DocsDocumentationsearchSearchrocket_launchGet startedInstallationaddFundamentalsaddFirst stepsaddcodeDevelopConceptsaddAPI referenceaddTutorialsaddQuick referenceremoveCheat sheetRelease notesremove2025202420232022202120202019Pre-release featuresRoadmapopen_in_newweb_assetDeployConceptsaddStreamlit Community CloudaddSnowflakeOther platformsaddschoolKnowledge baseFAQInstalling dependenci

In [7]:
#Splitting the documents

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 5000,
    chunk_overlap = 1000
)

documents_chunk = text_splitter.split_documents(docs)

In [8]:
#Tokenize and load the documents to the vector store
vector_db = Chroma.from_documents(
    documents=documents_chunk,
    embedding=OpenAIEmbeddings()
)

In [9]:
# Retrieve

def _get_context_retriever_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
        ("user", "Given the above conversation, generate a search query to look up in order to get inforamtion relevant to the conversation, focusing on the most recent messages."),
    ])
    retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

    return retriever_chain

In [10]:
def get_conversational_rag_chain(llm):
    retriever_chain = _get_context_retriever_chain(vector_db, llm)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
        """You are a helpful assistant. You will have to answer to user's queries.
        You will have some context to help with your answers, but now always would be completely related or helpful.
        You can also use your knowledge to assist answering the user's queries.\n
        {context}"""),
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
    ])
    stuff_documents_chain = create_stuff_documents_chain(llm, prompt)

    return create_retrieval_chain(retriever_chain, stuff_documents_chain)

In [12]:
# Augmented Generation

llm_stream_openai = ChatOpenAI(
    model="gpt-4o",
    temperature=0.3,
    streaming=True,
)

llm_stream_anthropic = ChatAnthropic(
    model="claude-3-5-sonnet-20240620",
    temperature=0.3,
    streaming=True,
)

llm_stream = llm_stream_openai  # Select between OpenAI and Anthropic models for the response

messages = [
    {"role": "user", "content": "Hi"},
    {"role": "assistant", "content": "Hi there! How can I assist you today?"},
    {"role": "user", "content": "What is the latest version of Streamlit?"},
    {"role": "user", "content": "What are the latest features on Streamlit?"},
]
messages = [HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"]) for m in messages]

conversation_rag_chain = get_conversational_rag_chain(llm_stream)
response_message = "*(RAG Response)*\n"
for chunk in conversation_rag_chain.pick("answer").stream({"messages": messages[:-1], "input": messages[-1].content}):
    response_message += chunk
    print(chunk, end="", flush=True)

messages.append({"role": "assistant", "content": response_message})

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


The latest version of Streamlit is 1.44.0, released on March 25, 2025. Here are some of the highlights and features of this release:

1. **Advanced Theming Options**: You can now customize the appearance of your app with an assortment of configuration options, allowing you to change fonts, colors, and roundness without using CSS.

2. **st.badge**: A new feature to insert a colored badge element. You can also include badges in Markdown using a new directive.

3. **streamlit init**: A command to create all the local files you need for a new Streamlit app.

Additionally, there are several notable changes and bug fixes, such as:

- **st.exception**: Now includes links to open Google or ChatGPT with the contents of the exception.
- **User's Locale Access**: You can access the user's locale through `st.context`.
- **Improved Performance**: Streamlit uses the React 18 `createRoot` API for its frontend to improve performance.
- **Compatibility with AI Tools**: Script compilation errors are log